In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
# Define image dimensions
img_height, img_width = 240, 240
batch_size = 32

# Load dataset using ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Split 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    "dataset/images",  # Replace with the path to your dataset
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use this for training data
)

validation_generator = datagen.flow_from_directory(
    "dataset/images",  # Replace with the path to your dataset
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use this for validation data
)

Found 1050 images belonging to 8 classes.
Found 259 images belonging to 8 classes.


In [3]:
# Load MobileNetV2 with pre-trained weights (excluding the top classification layer)
base_model = MobileNetV2(
    input_shape=(img_height, img_width, 3),  # Input shape for 240x240 RGB images
    include_top=False,  # Exclude the top classification layer
    weights='imagenet'  # Use pre-trained weights from ImageNet
)

# Freeze the base model to prevent training its layers initially
base_model.trainable = False

In [4]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer

# Combine the base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
model.compile(
    optimizer='adam',  # Adam optimizer
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy']  # Track accuracy during training
)

In [6]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Number of epochs
)

Epoch 1/10
32/32 [==============================] - 31s 921ms/step - loss: 0.4534 - accuracy: 0.8487 - val_loss: 0.3451 - val_accuracy: 0.9141
Epoch 2/10
32/32 [==============================] - 26s 832ms/step - loss: 0.0303 - accuracy: 0.9902 - val_loss: 0.4610 - val_accuracy: 0.8867
Epoch 3/10
32/32 [==============================] - 27s 833ms/step - loss: 0.0149 - accuracy: 0.9980 - val_loss: 0.4496 - val_accuracy: 0.9180
Epoch 4/10
32/32 [==============================] - 27s 834ms/step - loss: 0.0096 - accuracy: 0.9971 - val_loss: 0.5391 - val_accuracy: 0.9062
Epoch 5/10
32/32 [==============================] - 26s 809ms/step - loss: 0.0080 - accuracy: 0.9990 - val_loss: 0.5127 - val_accuracy: 0.9062
Epoch 6/10
32/32 [==============================] - 26s 813ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.4992 - val_accuracy: 0.9062
Epoch 7/10
32/32 [==============================] - 26s 822ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.5381 - val_accuracy: 0.9102

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.4f}")

9/9 [==============================] - 6s 655ms/step - loss: 0.4520 - accuracy: 0.8919
Validation Accuracy: 0.8919


In [9]:
# Unfreeze the top layers of the base model
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=5  # Additional epochs for fine-tuning
)

Epoch 1/5
28/28 [==============================] - 118s 4s/step - loss: 1.7504 - accuracy: 0.5582 - val_loss: 0.4355 - val_accuracy: 0.8973
Epoch 2/5
28/28 [==============================] - 102s 4s/step - loss: 0.3047 - accuracy: 0.9016 - val_loss: 0.4426 - val_accuracy: 0.9018
Epoch 3/5
28/28 [==============================] - 100s 4s/step - loss: 0.1060 - accuracy: 0.9687 - val_loss: 0.4121 - val_accuracy: 0.8973
Epoch 4/5
28/28 [==============================] - 100s 4s/step - loss: 0.0618 - accuracy: 0.9899 - val_loss: 0.4073 - val_accuracy: 0.8929
Epoch 5/5
28/28 [==============================] - 100s 4s/step - loss: 0.0577 - accuracy: 0.9866 - val_loss: 0.3913 - val_accuracy: 0.8973


In [8]:
# Load the trained model
model.save("Final_handSignModel.h5")
print("Model saved as mobilenetv2_sign_model.h5")

Model saved as mobilenetv2_sign_model.h5
